# 检查质量不平衡的反应

# 计算热力学数据

In [ ]:
import math
import re
import pandas as pd
from equilibrator_api import ComponentContribution, Q_
from pandas import ExcelWriter, ExcelFile
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# 定义常量
R = 8.31446261815324  # 气体常数,单位: J K^-1 mol^-1
T = 310.15  # 温度,单位: K
pH = 7.5  # 假设我们在标准生理pH条件下

cc = ComponentContribution()

# 读取Excel文件
excel_file = ExcelFile("6. cheshire gap/14067gem5.xlsx")
    
# 从xlsx文件读取数据
df = excel_file.parse('reactions')

# 初始化新列
df["ΔG'°"] = None
df["ΔG'm"] = None
df["K_eq"] = None
df["Is Balanced"] = None

# 定义函数用于转换反应方程式格式
def convert_reaction_formula(formula):
    def replace(match):
        coeff, compound = match.groups()
        compound = compound.split('_')[0]  # 去除后缀
        if coeff == '1':
            return f'seed:{compound}'
        else:
            return f'{coeff} seed:{compound}'
    
    # 处理反应物
    formula = re.sub(r'\((\d+(?:\.\d+)?)\) (\w+_\w+)', replace, formula)
    
    # 处理生成物
    formula = re.sub(r'=> \((\d+(?:\.\d+)?)\) (\w+_\w+)', r'=> \1 seed:\2', formula)
    formula = re.sub(r'_\w+', '', formula)  # 去除生成物后缀
    
    return formula

# 遍历每个反应方程式并计算热力学数据
for index, row in df.iterrows():
    reaction_formula = row["Equation"]
    
    try:
        # 转换反应方程式格式
        converted_formula = convert_reaction_formula(reaction_formula)
        
        # 解析转换后的反应方程式
        reaction = cc.parse_reaction_formula(converted_formula)
        
        # 检查反应是否平衡
        is_balanced = reaction.is_balanced()
        df.at[index, "Is Balanced"] = is_balanced
        
        if is_balanced:
            # 计算标准条件下的吉布斯自由能变化 ΔG'°
            dG0_prime = cc.standard_dg_prime(reaction)
            dG0_prime_nominal = dG0_prime.magnitude
            
            # 计算生理条件下的吉布斯自由能变化 ΔG'm
            dGm_prime = cc.physiological_dg_prime(reaction)
            dGm_prime_nominal = dGm_prime.magnitude
            
            # 确保 ΔG'° 为 J/mol 单位
            dG_prime_joules = dG0_prime_nominal * 1000
            s = dG_prime_joules.nominal_value
            
            # 计算平衡常数 K_eq
            K_eq = math.exp(-s / (R * T))
            
            # 将计算结果存储到相应的列中
            df.at[index, "ΔG'°"] = dG0_prime_nominal
            df.at[index, "ΔG'm"] = dGm_prime_nominal
            df.at[index, "K_eq"] = K_eq
        else:
            print(f"Reaction {converted_formula} is not balanced, skipping thermodynamic calculations.")
    except Exception as e:
        print(f"Error processing reaction: {converted_formula}")
        print(f"Error message: {str(e)}")
# 将所有工作表写入新的Excel文件
with ExcelWriter("6. cheshire gap/14067gem6.xlsx") as writer:
    for sheet_name in excel_file.sheet_names:
        if sheet_name == 'reactions':
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            excel_file.parse(sheet_name).to_excel(writer, sheet_name=sheet_name, index=False)

# 寻找无效atp循环

In [4]:
import cobra

# 1. 加载模型
updated_model = cobra.io.read_sbml_model('14067gem20_CGXII.xml')

#2. 将所有反应的通量限制在0到1之间
for reaction in updated_model.reactions:
    if reaction.reversibility:
        reaction.lower_bound = -1  # 如果反应可逆，设置反应的最小通量为-1
        reaction.upper_bound = 1  # 设置反应的最大通量为1
    else:
        reaction.lower_bound = 0  # 如果反应不可逆，设置反应的最小通量为0
        reaction.upper_bound = 1  # 设置反应的最大通量为1
 
#3. 关闭所有EX_开头的交换反应
for reaction in updated_model.reactions:
     if reaction.id.startswith('EX_'):
         # print(f"{reaction.id}")# 判断反应ID是否以EX_开头
         reaction.lower_bound = 0  # 将最小通量设置为0
         reaction.upper_bound = 0  # 将最大通量设置为0，关闭反应
        
reaction_ids_1 = {
}# 

# 确保反应在模型中
for reaction_id in reaction_ids_1:
    if reaction_id in updated_model.reactions:
        reaction = updated_model.reactions.get_by_id(reaction_id)
        
        # 打印原始上下限
        print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        
        # 设置新的上下限
        reaction.lower_bound = -1  # 设置下界
        reaction.upper_bound = 1 # 设置上界
        
        # 打印修改后的上下限
        print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
    else:
        print(f"反应 {reaction_id} 不存在于模型中！")

reaction_ids = {'rxn00062_c'}

# 确保反应在模型中
for reaction_id in reaction_ids:
    if reaction_id in updated_model.reactions:
        reaction = updated_model.reactions.get_by_id(reaction_id)
        
        # 打印原始上下限
        print(f"反应 {reaction_id} 原始上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
        
        # 设置新的上下限
        reaction.lower_bound = -1000  # 设置下界
        reaction.upper_bound = 1000  # 设置上界
        
        # 打印修改后的上下限
        print(f"反应 {reaction_id} 修改后上下限: {reaction.lower_bound} 至 {reaction.upper_bound}")
    else:
        print(f"反应 {reaction_id} 不存在于模型中！")
# 3. 设置ATPM为目标函数
updated_model.objective = 'rxn00062_c'  # 目标函数设置为ATPM

# 4. 进行FBA求解
solution = updated_model.optimize()

# 6. 检测是否存在能量生成循环（EGC）
def detect_egc(model, solution):
    fluxes = solution.fluxes.to_dict()
    
    # 打印所有通量不为0的反应，并输出反应式
    for reaction in model.reactions:
        # 获取反应的通量值，默认为0
        flux_value = fluxes.get(reaction.id, 0)
        if flux_value != 0:  # 如果通量不为0
            reaction_str = reaction.reaction  # 获取反应式
            print(f"Reaction: {reaction.id}, Flux: {flux_value}, Reaction equation: {reaction_str}")

    return fluxes

# 6. 输出检测到的EGC
egc_reactions = detect_egc(updated_model, solution)
if egc_reactions:
    print("检测到以下能量生成循环（EGC）反应：")
    # 如果需要，也可以打印出具体的反应信息或反应式
    # print(egc_reactions)
else:
    print("没有检测到能量生成循环（EGC）。")

# 7. 输出FBA结果
print("FBA结果：")
print(solution)

No objective coefficients in model. Unclear what should be optimized


反应 rxn00062_c 原始上下限: 0 至 1
反应 rxn00062_c 修改后上下限: -1000 至 1000
检测到以下能量生成循环（EGC）反应：
FBA结果：
<Solution 0.000 at 0x2875ac38bc0>


In [7]:
import cobra

# 1. 加载模型
updated_model = cobra.io.read_sbml_model('14067gem10.xml')

# 2. 设置反应的通量限制
for reaction in updated_model.reactions:
    if reaction.id == "rxnxxx_c":  # ATPM反应的通量范围设置为0到1000
        reaction.lower_bound = 0
        reaction.upper_bound = 1000
    elif reaction.id.startswith("EX_"):  # 关闭所有EX_开头的交换反应
        reaction.lower_bound = 0
        reaction.upper_bound = 0
    else:  # 其他反应的通量限制根据可逆性设置
        if reaction.reversibility:
            reaction.lower_bound = -1  # 如果反应可逆，设置反应的最小通量为-1
            reaction.upper_bound = 1  # 设置反应的最大通量为1
        else:
            print(reaction.reversibility)
            reaction.lower_bound = 0  # 如果反应不可逆，设置反应的最小通量为0
            reaction.upper_bound = 1  # 设置反应的最大通量为1

# 3. 设置ATPM为目标函数
updated_model.objective = 'rxnxxx_c'  # 目标函数设置为ATPM

# 4. 进行FBA求解
solution = updated_model.optimize()

# 5. 检测并输出所有通量不为0的反应（不包括ATPM）
def detect_non_atpm_reactions(model, solution):
    fluxes = solution.fluxes.to_dict()
    return {reaction.id: (fluxes[reaction.id], reaction.reaction) 
            for reaction in model.reactions 
            if fluxes.get(reaction.id, 0) != 0 and reaction.id != "ATPM"}

# 获取所有有通量的反应（不包括ATPM）
egc_reactions = detect_non_atpm_reactions(updated_model, solution)
if egc_reactions:
    print("检测到以下有通量的反应（不包括ATPM）：")
    for reaction_id, (flux_value, reaction_str) in egc_reactions.items():
        print(f"Reaction: {reaction_id}, Flux: {flux_value}, Reaction equation: {reaction_str}")
else:
    print("没有检测到有通量的反应（不包括ATPM）。")

# 6. 逐个关闭反应并检查ATPM的变化
def close_and_check_reaction(model, reaction_id, original_bounds):
    """关闭反应并检查ATP流量是否下降"""
    reaction = model.reactions.get_by_id(reaction_id)
    
    # 关闭当前反应的通量
    reaction.lower_bound = 0
    reaction.upper_bound = 0
    
    # 进行FBA求解
    solution_after_closing = model.optimize()
    
    # 获取关闭前后的ATPM流量
    atpm_flux_before = solution.objective_value
    atpm_flux_after = solution_after_closing.objective_value
    
    # 恢复反应的原始通量
    reaction.lower_bound = original_bounds[0]
    reaction.upper_bound = original_bounds[1]
    
    return atpm_flux_after < atpm_flux_before  # 如果ATPM流量下降返回True

# 逐个检查反应的影响并关闭
closed_reactions = []  # 用于记录关闭的反应

for reaction_id, (flux_value, reaction_str) in list(egc_reactions.items()):
    # 获取反应的原始通量限制
    reaction = updated_model.reactions.get_by_id(reaction_id)
    original_bounds = (reaction.lower_bound, reaction.upper_bound)
    
    # 关闭反应并检查是否影响ATPM的通量
    if close_and_check_reaction(updated_model, reaction_id, original_bounds):
        closed_reactions.append(reaction_id)
        print(f"关闭反应: {reaction_id} (ATPM通量下降)")

# 输出最终关闭的反应
print("\n最终关闭的反应（ATPM通量下降的反应）：")
print(closed_reactions)


No objective coefficients in model. Unclear what should be optimized


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [ ]:
import cobra

# 1. 加载模型
updated_model = cobra.io.read_sbml_model('14067gem10.xml')

# 2. 设置反应的通量限制
for reaction in updated_model.reactions:
    if reaction.id == "rxn00062_c":  # ATPM反应的通量范围设置为0到1000
        reaction.lower_bound = 0
        reaction.upper_bound = 1000
    elif reaction.id.startswith("EX_"):  # 关闭所有EX_开头的交换反应
        reaction.lower_bound = 0
        reaction.upper_bound = 0
    else:  # 其他反应的通量限制根据可逆性设置
        if reaction.reversibility:
            reaction.lower_bound = -1  # 如果反应可逆，设置反应的最小通量为-1
            reaction.upper_bound = 1  # 设置反应的最大通量为1
        else:
            print(reaction.reversibility)
            reaction.lower_bound = 0  # 如果反应不可逆，设置反应的最小通量为0
            reaction.upper_bound = 1 

# 3. 设置ATPM为目标函数        
updated_model.objective = 'rxn00062_c'  # 目标函数设置为ATPM

# 4. 进行FBA求解
solution = updated_model.optimize()
original_atpm_flux = solution.fluxes['rxn00062_c']
print(f"Original ATPM flux: {original_atpm_flux}")

# 5. 获取需要测试的反应集合
reaction_ids_to_test = [
     'rxn05465_c', 'rxn06106_c', 'rxn06109_c', 'rxn08019_c', 'rxn08444_c', 'rxn08766_c', 'rxn09448_c', 'rxn09458_c', 'rxn10030_c', 'rxn10042_c', 'rxn10043_c', 'rxn10954_c', 'rxn11676_c','rxn00097_c','rxn00799_c', 'rxn05289_c', 
]

# 6. 定义辅助函数
def check_atpm_flux(model):
    """检查rxn00062_c反应的通量值"""
    return model.reactions.get_by_id('rxn00062_c').flux  # 返回ATP的通量值

reactions_decreased_atpm = []  # 记录导致rxn00062_c通量下降的反应

# 7. 逐个关闭反应并检查rxn00062_c的变化
for reaction_id in reaction_ids_to_test:
    if reaction_id in updated_model.reactions:
        reaction = updated_model.reactions.get_by_id(reaction_id)

        # 关闭当前反应
        original_lb, original_ub = reaction.lower_bound, reaction.upper_bound
        reaction.lower_bound = 0
        reaction.upper_bound = 0
        
        # 进行FBA求解
        solution_after_closing = updated_model.optimize()
        atpm_flux_after = check_atpm_flux(updated_model) 

        if atpm_flux_after < original_atpm_flux:
            # rxn00062_c通量下降，记录该反应
            reactions_decreased_atpm.append(reaction_id)
            original_atpm_flux = atpm_flux_after  # 更新original_atpm_flux
            print(f"Reaction {reaction_id} decreased ATPM flux to {atpm_flux_after}. Keeping it closed.")
        else:
            # 通量没下降，恢复反应的原始上下限
            reaction.lower_bound = original_lb
            reaction.upper_bound = original_ub
            print(f"Reaction {reaction_id} did not decrease ATPM flux. Reverting to original bounds.")
    else:
        print(f"Reaction {reaction_id} does not exist in the model!")
        
# 8. 输出导致rxn00062_c通量下降的反应列表
print("\nReactions that decreased rxn00062_c flux when individually closed:")
print(reactions_decreased_atpm)

# 找到不参与反应的代谢物及基因

In [4]:
from cobra.io import read_sbml_model
import pandas as pd

def find_inactive_metabolites(model_path, output_file):
    # 读取SBML模型
    print("正在读取模型文件...")
    model = read_sbml_model(model_path)
    
    # 找出不参与反应的代谢物
    inactive_metabolites = []
    print("正在分析代谢物...")
    
    for metabolite in model.metabolites:
        # 检查代谢物参与的反应数量
        if len(metabolite.reactions) == 0:
            # 收集代谢物信息
            metabolite_info = {
                'Metabolite ID': metabolite.id,
                'Name': metabolite.name,
                'Formula': metabolite.formula if hasattr(metabolite, 'formula') else '',
                'Compartment': metabolite.compartment if hasattr(metabolite, 'compartment') else '',
                'Charge': metabolite.charge if hasattr(metabolite, 'charge') else '',
            }
            inactive_metabolites.append(metabolite_info)
    
    # 转换为DataFrame
    df = pd.DataFrame(inactive_metabolites)
    
    # 保存结果
    df.to_excel(output_file, index=False)
    
    # 打印统计信息
    total_metabolites = len(model.metabolites)
    inactive_count = len(inactive_metabolites)
    print(f"\n分析完成！")
    print(f"总代谢物数量: {total_metabolites}")
    print(f"不参与反应的代谢物数量: {inactive_count}")
    print(f"不参与反应的代谢物比例: {(inactive_count/total_metabolites)*100:.2f}%")
    print(f"\n结果已保存至: {output_file}")
    
    return df

# 使用示例
if __name__ == "__main__":
    model_path = "14067gem20_CGXII.xml"  # 你的XML文件路径
    output_file = "inactive_metabolites.xlsx"  # 输出文件名
    
    try:
        inactive_mets = find_inactive_metabolites(model_path, output_file)
    except Exception as e:
        print(f"发生错误: {str(e)}")

正在读取模型文件...


No objective coefficients in model. Unclear what should be optimized


正在分析代谢物...

分析完成！
总代谢物数量: 1620
不参与反应的代谢物数量: 1
不参与反应的代谢物比例: 0.06%

结果已保存至: inactive_metabolites.xlsx


# 删去不参与反应的代谢物及基因

In [7]:
from cobra.io import load_json_model,read_sbml_model
import pandas as pd

def find_inactive_genes(model_path, output_file):
    # 读取SBML模型
    print("正在读取模型文件...")
    model = read_sbml_model(model_path)

    # 找出不参与反应的基因
    inactive_genes = []
    print("正在分析基因...")

    for gene in model.genes:
        # 检查基因参与的反应数量
        if len(gene.reactions) == 0:
            # 收集基因信息
            gene_info = {
                'Gene ID': gene.id,
                'Name': gene.name if hasattr(gene, 'name') else '',
                'Associated Reactions': ', '.join([reaction.id for reaction in gene.reactions]) if gene.reactions else '',
            }
            inactive_genes.append(gene_info)

    # 转换为DataFrame
    df = pd.DataFrame(inactive_genes)

    # 保存结果
    df.to_excel(output_file, index=False)

    # 打印统计信息
    total_genes = len(model.genes)
    inactive_count = len(inactive_genes)
    print(f"\n分析完成！")
    print(f"总基因数量: {total_genes}")
    print(f"不参与反应的基因数量: {inactive_count}")
    print(f"不参与反应的基因比例: {(inactive_count/total_genes)*100:.2f}%")
    print(f"\n结果已保存至: {output_file}")

    return df

# 使用示例
if __name__ == "__main__":
    model_path = "core_output_core_model.xml"  # 你的XML文件路径
    output_file = "inactive_genes.xlsx"  # 输出文件名

    try:
        inactive_genes = find_inactive_genes(model_path, output_file)
    except Exception as e:
        print(f"发生错误: {str(e)}")


正在读取模型文件...


No objective coefficients in model. Unclear what should be optimized


正在分析基因...

分析完成！
总基因数量: 863
不参与反应的基因数量: 21
不参与反应的基因比例: 2.43%

结果已保存至: inactive_genes.xlsx


In [6]:
import pandas as pd

def remove_inactive_metabolites(inactive_file, model_file, output_file):
    try:
        # 读取不活跃代谢物的ID
        print("读取不活跃代谢物列表...")
        inactive_df = pd.read_excel(inactive_file)
        inactive_ids = set(inactive_df['Metabolite ID'])
        print(f"找到 {len(inactive_ids)} 个不活跃代谢物")

        # 读取原始模型文件的所有工作簿
        print(f"读取模型文件: {model_file}")
        excel_file = pd.ExcelFile(model_file)
        all_sheets = excel_file.sheet_names
        
        # 创建一个ExcelWriter对象来保存结果
        print(f"开始处理并保存到: {output_file}")
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            for sheet_name in all_sheets:
                # 读取每个工作簿
                df = pd.read_excel(model_file, sheet_name=sheet_name)
                
                # 如果是metabolites工作簿，进行过滤
                if sheet_name.lower() == 'metabolites':
                    original_count = len(df)
                    # 过滤掉不活跃代谢物
                    df = df[~df['metabolites_id'].isin(inactive_ids)]
                    removed_count = original_count - len(df)
                    print(f"在metabolites工作簿中移除了 {removed_count} 个代谢物")
                
                # 保存到新文件
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"已保存工作簿: {sheet_name}")
        
        print("处理完成！")
        
    except Exception as e:
        print(f"处理过程中发生错误: {str(e)}")

# 使用示例
if __name__ == "__main__":
    inactive_file = "inactive_metabolites.xlsx"  # 不活跃代谢物文件
    model_file = "model_history/14067gem14.xlsx"  # 原始模型文件
    output_file = "model_history/14067gem15.xlsx"  # 输出文件
    
    remove_inactive_metabolites(inactive_file, model_file, output_file)

读取不活跃代谢物列表...
找到 117 个不活跃代谢物
读取模型文件: 14067gem14.xlsx
开始处理并保存到: 14067gem15.xlsx
已保存工作簿: reactions
已保存工作簿: genes
在metabolites工作簿中移除了 117 个代谢物
已保存工作簿: metabolites
处理完成！


In [ ]:
import pandas as pd

def remove_inactive_genes(inactive_file, model_file, output_file):
    try:
        # 读取不活跃基因的ID
        print("读取不活跃基因列表...")
        inactive_df = pd.read_excel(inactive_file)
        inactive_ids = set(inactive_df['Gene ID'])
        print(f"找到 {len(inactive_ids)} 个不活跃基因")

        # 读取原始模型文件的所有工作簿
        print(f"读取模型文件: {model_file}")
        excel_file = pd.ExcelFile(model_file)
        all_sheets = excel_file.sheet_names
        
        # 创建一个ExcelWriter对象来保存结果
        print(f"开始处理并保存到: {output_file}")
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            for sheet_name in all_sheets:
                # 读取每个工作簿
                df = pd.read_excel(model_file, sheet_name=sheet_name)
                
                # 如果是genes工作簿，进行过滤
                if sheet_name.lower() == 'genes':
                    original_count = len(df)
                    # 过滤掉不活跃基因
                    df = df[~df['GeneID'].isin(inactive_ids)]
                    removed_count = original_count - len(df)
                    print(f"在genes工作簿中移除了 {removed_count} 个基因")
                
                # 保存到新文件
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"已保存工作簿: {sheet_name}")
        
        print("处理完成！")
        
    except Exception as e:
        print(f"处理过程中发生错误: {str(e)}")

# 使用示例
if __name__ == "__main__":
    inactive_file = "inactive_genes.xlsx"  # 不活跃基因文件
    model_file = "7.AMMEDEUS/14067gem8.xlsx"  # 原始模型文件
    output_file = "7.AMMEDEUS/14067gem9.xlsx"  # 输出文件
    
    remove_inactive_genes(inactive_file, model_file, output_file)


# 寻找断反应（存在瑕疵慎用）

In [4]:
import cobra
import pandas as pd

def find_dead_end_reactions(model):
    dead_end_reactions = []
    
    for reaction in model.reactions:
        if reaction.id.startswith('EX_') or reaction.id.startswith('Bio_'):
            continue
        
        if len(reaction.reactants) == 0 or len(reaction.products) == 0:
            dead_end_reactions.append(reaction)
            continue
        
        reactant_dead_end = True
        for reactant in reaction.reactants:
            if reactant in [m for r in model.reactions for m in r.products]:
                reactant_dead_end = False
                break
        
        product_dead_end = True        
        for product in reaction.products:
            if product in [m for r in model.reactions for m in r.reactants]:
                product_dead_end = False
                break
        
        if reactant_dead_end or product_dead_end:
            dead_end_reactions.append(reaction)
    
    return dead_end_reactions

def export_dead_end_reactions(dead_end_reactions, output_file):
    data = []
    for reaction in dead_end_reactions:
        reactants = [f"{coeff} {met.id}" for met, coeff in reaction.metabolites.items() if coeff < 0]
        products = [f"{coeff} {met.id}" for met, coeff in reaction.metabolites.items() if coeff > 0]
        data.append({
            'Reaction ID': reaction.id,
            'Reaction Name': reaction.name,
            'Reaction Formula': f"{' + '.join(reactants)} --> {' + '.join(products)}",
            'Gene Reaction Rule': reaction.gene_reaction_rule,
            'Lower Bound': reaction.lower_bound,
            'Upper Bound': reaction.upper_bound,
            'Reaction Notes': getattr(reaction,'notes','')
        })
    
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    print(f"已将 {len(dead_end_reactions)} 个断反应信息导出到: {output_file}")

# 使用示例
if __name__ == "__main__":
    sbml_file = '14067gem15.xml'
    output_file = 'dead_end_reactions.xlsx'
    
    model = cobra.io.read_sbml_model(sbml_file)
    
    all_dead_end_reactions = []
    
    while True:
        dead_end_reactions = find_dead_end_reactions(model)
        if not dead_end_reactions:
            break
        all_dead_end_reactions.extend(dead_end_reactions)
        model.remove_reactions(dead_end_reactions)
    
    export_dead_end_reactions(all_dead_end_reactions, output_file)
    print("断反应识别完成")

No objective coefficients in model. Unclear what should be optimized


已将 34 个断反应信息导出到: dead_end_reactions.xlsx
断反应识别完成


In [ ]:
import openpyxl

def remove_dead_end_reactions(model_file, dead_end_file, output_file):
    try:
        # 读取断反应的Reaction ID
        print(f"读取断反应列表: {dead_end_file}")
        dead_end_wb = openpyxl.load_workbook(dead_end_file)
        dead_end_ws = dead_end_wb.active
        dead_end_ids = set(cell.value for cell in dead_end_ws['A'])
        print(f"找到 {len(dead_end_ids)} 个断反应")

        # 读取原始模型文件
        print(f"读取模型文件: {model_file}")
        model_wb = openpyxl.load_workbook(model_file)
        
        # 处理reactions工作表
        print("处理reactions工作表...")
        reaction_ws = model_wb['reactions']
        to_delete = [row for row in reaction_ws.iter_rows(min_row=2) if row[0].value in dead_end_ids]
        for row in reversed(to_delete):
            reaction_ws.delete_rows(row[0].row)
        print(f"在reactions工作表中移除了 {len(to_delete)} 个反应")
        
        # 保存结果到新文件  
        print(f"保存结果到: {output_file}")
        model_wb.save(output_file)
        
        print("处理完成!")

    except Exception as e:
        print(f"处理过程中发生错误: {str(e)}")

# 使用示例        
if __name__ == "__main__":
    model_file = "7.AMMEDEUS/14067gem8.xlsx"  # 原始模型文件
    dead_end_file = "dead_end_reactions.xlsx"  # 断反应列表文件 
    output_file = "7.AMMEDEUS/14067gem9.xlsx"  # 输出文件
    
    remove_dead_end_reactions(model_file, dead_end_file, output_file)